In [ ]:
labels_to_names= {0:'class_name1', 1:'class_name2', 2:'class_name3'}

In [ ]:
from utils.image import read_image_bgr, preprocess_image, resize_image
from utils.visualization import draw_box, draw_caption
from utils.colors import label_color

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = read_image_bgr('test.JPG')
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)
batch = np.expand_dims(image, axis=0)

In [ ]:
import json
import numpy
import requests
data = json.dumps({"signature_name": "predict",
                   "instances": batch.tolist()})
headers = {"content-type": "application/json"}
json_response = requests.post('http://172.16.8.252:8501/v1/models/retinanet:predict',
                              data=data, headers=headers)
new_predictions = json.loads(json_response.text)["predictions"]

In [ ]:
boxes = np.array(new_predictions[0]['output1'])
scores = np.array(new_predictions[0]['output2'])
labels = np.array(new_predictions[0]['output3'])

In [ ]:
# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes, scores, labels):
    # scores are sorted so we can break
    if score < 0.5:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    print(labels_to_names[label], score)
    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)
    
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()